In [20]:
# ~ https://www.gutenberg.org/files/40796/40796-0.txt
from sklearn.feature_extraction.text import TfidfVectorizer
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import pickle

In [6]:
page = requests.get('https://www.gutenberg.org/files/40796/40796-h/40796-h.htm')
soup = BeautifulSoup(page.content, 'html.parser')
txt=soup.get_text()
# result = requests.get('https://www.gutenberg.org/files/40796/40796-0.txt')
# txt = result.content
# print(txt[:100])







     The Project Gutenberg eBook of Foods and Household Management by Helen Kinne and Anna M.


In [14]:
# text = re.sub(r'\n\s*\n', r' ', txt.strip(), flags=re.M)
text = re.sub(r'\r', r'', txt, flags=re.M)
text_array = re.split(r'\n\s*\n', text.strip(), flags=re.M)
text_array

['The Project Gutenberg eBook of Foods and Household Management by Helen Kinne and Anna M. Cooley.',
 'The Project Gutenberg EBook of Foods and Household Management, by \nHelen Kinne and Anna M. Cooley',
 'This eBook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  You may copy it, give it away or\nre-use it under the terms of the Project Gutenberg License included\nwith this eBook or online at www.gutenberg.org',
 'Title: Foods and Household Management\n       A Textbook of the Household Arts',
 'Author: Helen Kinne\n        Anna M. Cooley',
 'Release Date: September 18, 2012 [EBook #40796]',
 'Language: English',
 'Character set encoding: UTF-8',
 '*** START OF THIS PROJECT GUTENBERG EBOOK FOODS AND HOUSEHOLD MANAGEMENT ***',
 'Produced by Juliet Sutherland, Henry Gardiner and the\nOnline Distributed Proofreading Team at http://www.pgdp.net',
 'Transcriber’s Note: The original publication has been replicated faithfully except as listed\nhere.

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
countVec = CountVectorizer(max_features= 5000, stop_words='english', min_df=.01, max_df=.90)

#%%
#use CountVectorizer.fit(self, raw_documents[, y] to learn vocabulary dictionary of all tokens in raw documents
#raw documents in this case will betweetsFrameWords["Text"] (processed text)
countVec.fit(text_array)
#useful debug, get an idea of the item list you generated
list(countVec.vocabulary_.items())

#%%
#convert to bag of words
#sparse matrix representation? (README: could use an edit/explanation)
countVec_count = countVec.transform(text_array)

#%%
#make array from number of occurrences
occ = np.asarray(countVec_count.sum(axis=0)).ravel().tolist()

#make a new data frame with columns term and occurrences, meaning word and number of occurences
bowListFrame = pd.DataFrame({'term': countVec.get_feature_names_out(), 'occurrences': occ})
# print(bowListFrame)

#sort in order of number of word occurences, most->least. if you leave of ascending flag should default ASC
bowListFrame.sort_values(by='occurrences', ascending=False).head(60)

#%%
#now, convert to a more useful ranking system, tf-idf weights
#TfidfTransformer: scale raw word counts to a weighted ranking using the
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html
foodWordTransformer = TfidfTransformer()

#initial fit representation using transformer object
tweetWeights = foodWordTransformer.fit_transform(countVec_count)

#follow similar process to making new data frame with word occurrences, but with term weights
tweetWeightsFin = np.asarray(tweetWeights.mean(axis=0)).ravel().tolist()

#now that we've done Tfid, make a dataframe with weights and names
tweetWeightFrame = pd.DataFrame({'term': countVec.get_feature_names_out(), 'weight': tweetWeightsFin})
# print(tweetWeightFrame)
tweetWeightFrame.sort_values(by='weight', ascending=False).head(20)

,term,weight
482,fig,0.036670
1260,water,0.029040
330,courtesy,0.027108
511,food,0.024735
748,milk,0.021992
572,gutenberg,0.020680
914,project,0.018466
1228,used,0.017958
173,bread,0.017918
734,meat,0.017669


In [24]:
tfidf_vec = TfidfVectorizer()
transformed = tfidf_vec.fit_transform(text_array)

In [33]:
# get the first vector out (for the first document) 
first_vector_tfidfvectorizer=transformed[0] 

# place tf-idf values in a pandas data frame 
df = pd.DataFrame(first_vector_tfidfvectorizer.T,
                  index=tfidf_vec.get_feature_names_out(), 
                  columns=["tfidf"]
                  )
df.sort_values(by=["tfidf"],ascending=False)

,tfidf
cooley,0.369842
kinne,0.369842
helen,0.369842
anna,0.351669
ebook,0.337573
...,...
dressmaking,0.000000
dressings,0.000000
dressing2,0.000000
dressing,0.000000


In [25]:
index_value={i[1]:i[0] for i in tfidf_vec.vocabulary_.items()}

In [27]:
fully_indexed = []
transformed = np.array(transformed.todense())
for row in transformed:
    fully_indexed.append({index_value[column]:value for (column,value) in enumerate(row)})


In [30]:
len(fully_indexed)

769

In [29]:
tweetWeightFrame2 = pd.DataFrame({'term': tfidf_vec.vocabulary_, 'weight': fully_indexed})
print(tweetWeightFrame2)
tweetWeightFrame2.sort_values(by='weight', ascending=False).head(20)

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

{'the': 7564,
 'project': 5942,
 'gutenberg': 3821,
 'ebook': 2879,
 'of': 5261,
 'foods': 3460,
 'and': 890,
 'household': 4024,
 'management': 4766,
 'by': 1536,
 'helen': 3939,
 'kinne': 4417,
 'anna': 910,
 'cooley': 2179,
 'this': 7627,
 'is': 4321,
 'for': 3466,
 'use': 7952,
 'anyone': 919,
 'anywhere': 921,
 'at': 1042,
 'no': 5166,
 'cost': 2228,
 'with': 8273,
 'almost': 844,
 'restrictions': 6354,
 'whatsoever': 8189,
 'you': 8370,
 'may': 4841,
 'copy': 2188,
 'it': 4329,
 'give': 3685,
 'away': 1086,
 'or': 5344,
 're': 6143,
 'under': 7864,
 'terms': 7545,
 'license': 4582,
 'included': 4137,
 'online': 5318,
 'www': 8333,
 'org': 5364,
 'title': 7683,
 'textbook': 7554,
 'arts': 1008,
 'author': 1067,
 'release': 6246,
 'date': 2416,
 'september': 6721,
 '18': 178,
 '2012': 229,
 '40796': 479,
 'language': 4469,
 'english': 2999,
 'character': 1741,
 'set': 6733,
 'encoding': 2979,
 'utf': 7966,
 'start': 7160,
 'produced': 5920,
 'juliet': 4377,
 'sutherland': 7426,
 'h

In [42]:
def calculate_tfidf_via_transformer(docs):
    #instantiate CountVectorizer() 
    cv=CountVectorizer() 

    # this steps generates word counts for the words in your docs 
    word_count_vector=cv.fit_transform(docs)
    tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
    tfidf_transformer.fit(word_count_vector)
    # print idf values 
    df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names_out(),columns=["idf_weights"]) 

    # sort ascending 
    # df_idf.sort_values(by=['idf_weights'])
    # count matrix 
    count_vector=cv.transform(docs) 

    # tf-idf scores 
    tf_idf_vector=tfidf_transformer.transform(count_vector)
    
    return cv, tfidf_transformer, tf_idf_vector

def print_tfidf_via_transformer(docs):
    
    cv, tfidf_transformer, tf_idf_vector = calculate_tfidf_via_transformer(docs=docs)
    
    feature_names = cv.get_feature_names_out() 

    #get tfidf vector for first document 
    first_document_vector=tf_idf_vector[0] 

    #print the scores which includes 0's for words in vocab from other docs that arent included 
    df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"]) 
    return df.sort_values(by=["tfidf"],ascending=False)

In [47]:
def calculate_tfidf_via_vectorizer(docs):

    # settings that you use for count vectorizer will go here 
    tfidf_vectorizer=TfidfVectorizer(use_idf=True) 

    # just send in all your docs here 
    tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(docs)
    
    return tfidf_vectorizer, tfidf_vectorizer_vectors
    
def print_tfidf_via_vectorizer(docs):
    
    tfidf_vectorizer, tfidf_vectorizer_vectors = calculate_tfidf_via_vectorizer(docs)
    
    # get the first vector out (for the first document) 
    first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[0]

    # place tf-idf values in a pandas data frame 
    df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names_out(), columns=["tfidf"])
    return df.sort_values(by=["tfidf"],ascending=False)

In [44]:
docs = ["the house had a tiny little mouse", 
"the cat saw the mouse", 
"the mouse ran away from the house", 
"the cat finally ate the mouse", 
"the end of the mouse story" 
]

In [45]:
print_tfidf_via_transformer(text_array)

,tfidf
cooley,0.369842
kinne,0.369842
helen,0.369842
anna,0.351669
ebook,0.337573
...,...
dressmaking,0.000000
dressings,0.000000
dressing2,0.000000
dressing,0.000000


In [48]:
print_tfidf_via_vectorizer(text_array)

,tfidf
cooley,0.369842
kinne,0.369842
helen,0.369842
anna,0.351669
ebook,0.337573
...,...
dressmaking,0.000000
dressings,0.000000
dressing2,0.000000
dressing,0.000000


In [49]:
tfidf_vectorizer, tfidf_vectorizer_vectors = calculate_tfidf_via_vectorizer(text_array)

In [50]:
with open('tfidf_vectorizer.pk', 'wb') as fin:
    pickle.dump(tfidf_vectorizer, fin)

In [ ]:
# now use this vectorizer to transform docs (Food category names) and 